In [109]:
#install required libraries
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd


import pandas as pd
import numpy as np
#data visualization packages
import matplotlib.pyplot as plt
#keras packages
import keras
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dropout
#model evaluation packages
from sklearn.metrics import f1_score, roc_auc_score, log_loss
from sklearn.model_selection import cross_val_score, cross_validate


from sklearn.preprocessing import normalize 
from keras.utils import np_utils

In [198]:
dataset = pd.read_csv('/content/breast-cancer.data', 
                  names=["Class", "age", "menopause", "tumor-size", "inv-nodes","node-caps","deg-malig","breast","breast-quad","irradiat"])
dataset.head()

,Class,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat
0,no-recurrence-events,30-39,premeno,30-34,0-2,no,3,left,left_low,no
1,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,right,right_up,no
2,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,left,left_low,no
3,no-recurrence-events,60-69,ge40,15-19,0-2,no,2,right,left_up,no
4,no-recurrence-events,40-49,premeno,0-4,0-2,no,2,right,right_low,no


In [199]:
dataset['Class'].describe()

count                      286
unique                       2
top       no-recurrence-events
freq                       201
Name: Class, dtype: object

In [200]:
#Q1 

dataset.loc[dataset["age"]=="20-29","newAge"]=1
dataset.loc[dataset["age"]=="30-39","newAge"]=2
dataset.loc[dataset["age"]=="40-49","newAge"]=3
dataset.loc[dataset["age"]=="50-59","newAge"]=4
dataset.loc[dataset["age"]=="60-69","newAge"]=5
dataset.loc[dataset["age"]=="70-79","newAge"]=6
dataset.drop('age',axis=1,inplace=True)


#no = 0
#yes = 1
dataset.loc[dataset["node-caps"]=="no","node_caps"]=0
dataset.loc[dataset["node-caps"]=="yes","node_caps"]=1
dataset.drop('node-caps',axis=1,inplace=True)


In [201]:
#Target Collumn
# no-recurrence-event = no breast cancer = 0
# recurrence event = breast cancer = 1

dataset.loc[dataset["Class"]=="no-recurrence-events",".Class"]=0
dataset.loc[dataset["Class"]=="recurrence-events",".Class"]=1
dataset.drop('Class',axis=1,inplace=True)

In [202]:
len(dataset['tumor-size'].unique())

11

In [203]:
def ownfunc(x):
  j=1
  for i in dataset[x].unique():
      dataset.loc[dataset[x]== i,('.'+ x)]= j
      j+=1
  dataset.drop(x,axis=1,inplace=True)


In [204]:
ownfunc('tumor-size')

In [205]:
ownfunc('inv-nodes')

In [206]:
dataset.head()

,menopause,deg-malig,breast,breast-quad,irradiat,newAge,node_caps,.Class,.tumor-size,.inv-nodes
0,premeno,3,left,left_low,no,2.0,0.0,0.0,1.0,1.0
1,premeno,2,right,right_up,no,3.0,0.0,0.0,2.0,1.0
2,premeno,2,left,left_low,no,3.0,0.0,0.0,2.0,1.0
3,ge40,2,right,left_up,no,5.0,0.0,0.0,3.0,1.0
4,premeno,2,right,right_low,no,3.0,0.0,0.0,4.0,1.0


In [207]:
dataset.drop('breast-quad',axis=1,inplace=True)

In [209]:
menopause = pd.get_dummies(dataset['menopause'])
breast = pd.get_dummies(dataset['breast'])
irradiat = pd.get_dummies(dataset['irradiat'])

In [210]:
dataset['menopause'].unique()

array(['premeno', 'ge40', 'lt40'], dtype=object)

In [212]:
dataset = dataset.join(menopause)
dataset = dataset.join(breast)
dataset = dataset.join(irradiat)

In [213]:
dataset.head()

,menopause,deg-malig,breast,irradiat,newAge,node_caps,.Class,.tumor-size,.inv-nodes,ge40,lt40,premeno,left,right,no,yes
0,premeno,3,left,no,2.0,0.0,0.0,1.0,1.0,0,0,1,1,0,1,0
1,premeno,2,right,no,3.0,0.0,0.0,2.0,1.0,0,0,1,0,1,1,0
2,premeno,2,left,no,3.0,0.0,0.0,2.0,1.0,0,0,1,1,0,1,0
3,ge40,2,right,no,5.0,0.0,0.0,3.0,1.0,1,0,0,0,1,1,0
4,premeno,2,right,no,3.0,0.0,0.0,4.0,1.0,0,0,1,0,1,1,0


In [214]:
dataset = dataset.drop('menopause',axis = 1)
dataset = dataset.drop('breast',axis = 1)
dataset = dataset.drop('irradiat',axis = 1)

In [215]:
dataset.head()

,deg-malig,newAge,node_caps,.Class,.tumor-size,.inv-nodes,ge40,lt40,premeno,left,right,no,yes
0,3,2.0,0.0,0.0,1.0,1.0,0,0,1,1,0,1,0
1,2,3.0,0.0,0.0,2.0,1.0,0,0,1,0,1,1,0
2,2,3.0,0.0,0.0,2.0,1.0,0,0,1,1,0,1,0
3,2,5.0,0.0,0.0,3.0,1.0,1,0,0,0,1,1,0
4,2,3.0,0.0,0.0,4.0,1.0,0,0,1,0,1,1,0


In [221]:
dataset.shape

(286, 13)

In [ ]:
#2

In [218]:
a = dataset.drop('.Class',axis=1)
X = a
y = dataset['.Class']


In [219]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 101)

In [220]:
#3
#feature scaling
from sklearn.preprocessing import StandardScaler
Standardscaler = StandardScaler()
#fit and transform training dataset
X_train = Standardscaler.fit_transform(X_train)
#transform testing dataset
X_test = Standardscaler.transform(X_test)

In [ ]:
#4

In [223]:
# Initialising the ANN
classifier = Sequential()
#units: It's an art, comes by experience, input+output/2
#initializer: how your weights are updated
#relu activation function
#input_dim: input shape (shape of the data, how many features do we have in the data)
# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 32, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))
#input_dim --> input dimensions
#activation --> relu activation
#kernel_initializer --> 'uniform' how the weights are updated uniformly
#avg(input_dim + output_dim) = 5.5

# Adding the second hidden layer
classifier.add(Dense(units = 32, kernel_initializer = 'uniform', activation = 'relu')) 

# Adding the third  hidden layer
classifier.add(Dense(units = 32, kernel_initializer = 'uniform', activation = 'relu')) 

# Adding the forth  hidden layer
classifier.add(Dense(units = 32, kernel_initializer = 'uniform', activation = 'relu')) 

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid')) 

In [224]:
# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
#multi class classification: categorical_crossentropy, sparsecategorical_crossentropy
#y_train[0]
# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 50,epochs = 20)

Epoch 1/20
4/4 [==============================] - 1s 5ms/step - loss: nan - accuracy: 0.6950
Epoch 2/20
4/4 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.7250
Epoch 3/20
4/4 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.7250
Epoch 4/20
4/4 [==============================] - 0s 5ms/step - loss: nan - accuracy: 0.7250
Epoch 5/20
4/4 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.7250
Epoch 6/20
4/4 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.7250
Epoch 7/20
4/4 [==============================] - 0s 5ms/step - loss: nan - accuracy: 0.7250
Epoch 8/20
4/4 [==============================] - 0s 5ms/step - loss: nan - accuracy: 0.7250
Epoch 9/20
4/4 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.7250
Epoch 10/20
4/4 [==============================] - 0s 5ms/step - loss: nan - accuracy: 0.7250
Epoch 11/20
4/4 [==============================] - 0s 4ms/step - loss

In [227]:
#evaluate the model for testing dataset
test_loss = classifier.evaluate(X_test, y_test, verbose=0)

#create evaluation dataframe
stats = pd.DataFrame({'Test accuracy' :  round(test_loss[1]*100,3),

                      'Total Loss'    : round(test_loss[0],3)}, index=[0])
#print evaluation dataframe
display(stats)

,Test accuracy,Total Loss
0,65.116,NaN


In [ ]:
#5


In [234]:
# Initialising the ANN
classifier2 = Sequential()
#units: It's an art, comes by experience, input+output/2
#initializer: how your weights are updated
#relu activation function
#input_dim: input shape (shape of the data, how many features do we have in the data)
# Adding the input layer and the first hidden layer
classifier2.add(Dense(units = 32, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))
#input_dim --> input dimensions
#activation --> relu activation
#kernel_initializer --> 'uniform' how the weights are updated uniformly
#avg(input_dim + output_dim) = 5.5

# Adding the second hidden layer
classifier2.add(Dense(units = 32, kernel_initializer = 'uniform', activation = 'relu')) 

# Adding the third  hidden layer
classifier2.add(Dense(units = 32, kernel_initializer = 'uniform', activation = 'relu')) 

# Adding the forth  hidden layer
classifier2.add(Dense(units = 32, kernel_initializer = 'uniform', activation = 'relu')) 

# Adding the fifth hidden layer
classifier2.add(Dropout(0.3)) 

# Adding the output layer
classifier2.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid')) 

In [235]:
# Compiling the ANN
classifier2.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
#multi class classification: categorical_crossentropy, sparsecategorical_crossentropy
#y_train[0]
# Fitting the ANN to the Training set
classifier2.fit(X_train, y_train, batch_size = 50,epochs = 20)

Epoch 1/20
4/4 [==============================] - 1s 4ms/step - loss: nan - accuracy: 0.7150
Epoch 2/20
4/4 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.7250
Epoch 3/20
4/4 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.7250
Epoch 4/20
4/4 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.7250
Epoch 5/20
4/4 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.7250
Epoch 6/20
4/4 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.7250
Epoch 7/20
4/4 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.7250
Epoch 8/20
4/4 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.7250
Epoch 9/20
4/4 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.7250
Epoch 10/20
4/4 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.7250
Epoch 11/20
4/4 [==============================] - 0s 4ms/step - loss

In [236]:
#evaluate the model for testing dataset
test_loss = classifier2.evaluate(X_test, y_test, verbose=0)

#create evaluation dataframe
stats = pd.DataFrame({'Test accuracy' :  round(test_loss[1]*100,3),

                      'Total Loss'    : round(test_loss[0],3)}, index=[0])
#print evaluation dataframe
display(stats)

,Test accuracy,Total Loss
0,65.116,NaN
